In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import cv2

from data.create_dataset import create_dataset

dataset_configs = json.load(open('data/dataset_configs.json', 'r'))
print(dataset_configs.keys())

In [ ]:
from model.directsam import DirectSAM

# model = DirectSAM(
#     "chendelong/DirectSAM-1800px-0424",
#     resolution=1800,
#     device='cuda'
#     )

model = None

In [ ]:
def visualize_colored_label_map(label_map, image):
    
    image = np.array(image)
    canvas = np.zeros((label_map.shape[0], label_map.shape[1], 3))

    for label_id in np.unique(label_map):
        mask = label_map == label_id
        canvas[mask] = np.mean(image[mask], axis=0)

    canvas = np.clip(canvas * 1.1, 0, 255)
    return canvas.astype(np.uint8) 

def visualize_image_with_contour(contour, image):

    image = np.array(image)
    image[contour] = [255, 255, 255]
    return image

In [ ]:


keys = ['DSA_merged']

for key in keys:
    config = dataset_configs[key]
    for split in ['train']:# , 'validation']:

        dataset = create_dataset(config, split, 1800, thickness=9)

        print(key, split, len(dataset), len(dataset.image_paths))
        print(config)
        print(f'\t"{key}" # {len(dataset)}')

        for index in range(10):
            index = random.randint(0, len(dataset)-1)
            sample = dataset[index]

            image = sample['image']
            label = sample['label']
            label_map = sample['label_map']

            if config['type'] != 'DSA':
                image_path = dataset.image_paths[index]
                assert os.path.exists(image_path)

            plt.figure(figsize=(24, 6))

            plt.subplot(1, 4, 1)
            plt.title('Label Map')
            plt.imshow(visualize_colored_label_map(label_map, image))
            plt.axis('off')

            plt.subplot(1, 4, 2)
            plt.title('Label Contour')
            plt.imshow(visualize_image_with_contour(label, image))
            plt.axis('off')

            if model is None:
                plt.subplot(1, 4, 3)
                plt.title('Label')
                plt.imshow(label, cmap='Greens', alpha=0.5)
                plt.axis('off')
            else:

                probs = model(image)

                plt.subplot(1, 4, 3)
                plt.title('Label and Pseudo Label')
                pseudo_label = probs > 0.5
                pseudo_label = cv2.resize(pseudo_label.astype(np.uint8), (label.shape[1], label.shape[0]), interpolation=cv2.INTER_NEAREST)

                plt.imshow(pseudo_label, cmap='Reds')
                plt.imshow(label, cmap='Greens', alpha=0.5)
                plt.axis('off')

                plt.subplot(1, 4, 4)
                plt.title('DirectSAM Probabilities')
                plt.imshow(probs, cmap='Reds')
                plt.axis('off')



            plt.show()